In [ ]:
%pip install tensorflow
%pip install --force-reinstall numpy==1.19.0
%pip install opencv-python

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
import cv2
import numpy as np
from numpy.linalg import norm
import os

# Load pre-trained VGG16 model with ImageNet weights
model = VGG16(weights='imagenet', include_top=False)

# Load and preprocess the image from a local file path
def preprocess_image(image_path):
    img = cv2.imread(image_path)  # Read image from the local file path
    if img is None:
      raise Exception(f"Unable to load image at path: {image_path}")
    
    img = cv2.resize(img, (224, 224))  # Resize to 224x224
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    img = preprocess_input(img)  # Normalize for the CNN
    return img

# Function to describe the difference based on the distance
def describe_difference(distance):
  if distance < 100:
    return "The images are very similar, with almost no noticeable differences."
  elif 100 <= distance < 500:
    return "The images have some noticeable differences, but they are still quite similar."
  elif 500 <= distance < 1000:
    return "The images have several differences, and they appear moderately distinct from one another."
  else:
    return "The images are very different, with significant changes in content or appearance."


def cnn_process(image_path1, image_path2) :
  # Preprocess the images
  image1 = preprocess_image(image_path1)
  image2 = preprocess_image(image_path2)

  # Extract features from the last convolutional layer
  features1 = model.predict(image1)
  features2 = model.predict(image2)

  # Flatten the feature vectors
  features1 = features1.flatten()
  features2 = features2.flatten()

  # Calculate Euclidean distance
  distance = norm(features1 - features2)
  # print(f'Euclidean Distance between the images: {distance}')
  print(f"{distance}")
  os.makedirs("output/txt", exist_ok=True)
  with open("output/txt/output_cropped_video.txt", "a") as file:
    file.write(str(distance) + "\n")  # Add a newline after each result

  # Get a sentence describing the difference
  # description = describe_difference(distance)
  # print(description)

In [16]:
import cv2
import os

def extract_and_save_scenes(video_path, output_dir):
    """
    Extracts one frame per second from the provided video, saves them as image files locally,
    and returns scenes as a dictionary.

    Parameters:
    video_path (string): Path of the video to be processed.
    output_dir (string): Directory where the extracted frames will be saved.

    Returns:
    list of dict: Each dictionary corresponds to a scene, containing:
        - index (int): Scene index
        - img_file (string): File name of the saved image
    """
    
    if not os.path.exists(video_path):
        print(f"{video_path}: File not found.")
        return []

    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Open the video file
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    if fps <= 0:
        print(f"Error: Could not retrieve FPS from {video_path}")
        return []

    scenes = []
    index = 0
    success, frame = cap.read()
    count = 0

    while success:
        # Capture frame every second (approximately)
        if count % int(fps) == 0:
            # Convert the input image to gray scale
            # frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            # Define the file name for the saved image
            img_file = os.path.join(output_dir, f"scene_{index}.jpg")
            
            # Save the frame as an image file
            cv2.imwrite(img_file, frame)
            
            # Append scene info to the list
            scenes.append({
                "index": index,
                "img_file": img_file
            })
            
            index += 1
        
        success, frame = cap.read()
        count += 1

    cap.release()
    
    return scenes

In [ ]:
import os

def compare_scenes_in_reverse(scenes, cnn_process):
    """
    Compares all scenes in reverse order using the cnn_process function.

    Parameters:
    scenes (list of dict): List of dictionaries where each dict contains:
        - index (int): Scene index
        - img_file (string): File name of the saved image
    cnn_process (function): Function to compare two images.

    Returns:
    None
    """

    # Compare images in reverse order
    for i in range(len(scenes) - 1, 0, -1):
        image_path1 = scenes[i]["img_file"]
        image_path2 = scenes[i - 1]["img_file"]
        # print(f"Comparing Scene {scenes[i]['index']} and Scene {scenes[i-1]['index']}:")
        # print(f"{scenes[i]['index']}-{scenes[i-1]['index']}:")
        cnn_process(image_path1, image_path2)

# Example Usage:
# Assuming 'scenes' is the list returned by extract_and_save_scenes() function
# and cnn_process() is the function to compare two images.
scenes = extract_and_save_scenes("data/videos/croppedvideo.mp4", "segmented_frames_cropped_video")
compare_scenes_in_reverse(scenes, cnn_process)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
159.17959594726562
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
672.5914916992188
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
498.52386474609375
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
142.71633911132812
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1548.2294921875
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
660.6053466796875
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
94.87960052490234
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
38.92616653442383
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1940.617431640625
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
74.10922241210938
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step


In [ ]:
video_path = "data/videos/croppedvideo.mp4" # Replace this with your actual video file path
output_dir = "segmented_frames_cropped_video"
scenes = extract_and_save_scenes(video_path, output_dir)
print(f"Extracted and saved {len(scenes)} scenes.")
compare_scenes_in_reverse(scenes, cnn_process)

Extracted and saved 29 scenes.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
159.17959594726562
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
672.5914916992188
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
498.52386474609375
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
142.71633911132812
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1548.2294921875
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
660.6053466796875
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
94.87960052490234
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
38.92616653442383
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1940.617431640625
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
74.10922241210938
1/1 ━━━━